---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

In [2]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

# Assignment 4 - Hypothesis Testing
This assignment requires more individual learning than previous assignments - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

Definitions:
* A _quarter_ is a specific three month period, Q1 is January through March, Q2 is April through June, Q3 is July through September, Q4 is October through December.
* A _recession_ is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
* A _recession bottom_ is the quarter within a recession which had the lowest GDP.
* A _university town_ is a city which has a high percentage of university students compared to the total population of the city.

**Hypothesis**: University towns have their mean housing prices less effected by recessions. Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (`price_ratio=quarter_before_recession/recession_bottom`)

The following data files are available for this assignment:
* From the [Zillow research data site](http://www.zillow.com/research/data/) there is housing data for the United States. In particular the datafile for [all homes at a city level](http://files.zillowstatic.com/research/public/City/City_Zhvi_AllHomes.csv), ```City_Zhvi_AllHomes.csv```, has median home sale prices at a fine grained level.
* From the Wikipedia page on college towns is a list of [university towns in the United States](https://en.wikipedia.org/wiki/List_of_college_towns#College_towns_in_the_United_States) which has been copy and pasted into the file ```university_towns.txt```.
* From Bureau of Economic Analysis, US Department of Commerce, the [GDP over time](http://www.bea.gov/national/index.htm#gdp) of the United States in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file ```gdplev.xls```. For this assignment, only look at GDP data from the first quarter of 2000 onward.

Each function in this assignment below is worth 10%, with the exception of ```run_ttest()```, which is worth 50%.

In [3]:
# Use this dictionary to map state names to two letter acronyms
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska', 'UT': 'Utah', 'OR': 'Oregon', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands', 'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [4]:
city_all_homes = pd.read_csv('City_Zhvi_AllHomes.csv')
city_all_homes.head()

,RegionID,RegionName,State,Metro,CountyName,SizeRank,1996-04,1996-05,1996-06,1996-07,...,2015-11,2015-12,2016-01,2016-02,2016-03,2016-04,2016-05,2016-06,2016-07,2016-08
0,6181,New York,NY,New York,Queens,1,NaN,NaN,NaN,NaN,...,573600,576200,578400,582200,588000,592200,592500,590200,588000,586400
1,12447,Los Angeles,CA,Los Angeles-Long Beach-Anaheim,Los Angeles,2,155000.0,154600.0,154400.0,154200.0,...,558200,560800,562800,565600,569700,574000,577800,580600,583000,585100
2,17426,Chicago,IL,Chicago,Cook,3,109700.0,109400.0,109300.0,109300.0,...,207800,206900,206200,205800,206200,207300,208200,209100,211000,213000
3,13271,Philadelphia,PA,Philadelphia,Philadelphia,4,50000.0,49900.0,49600.0,49400.0,...,122300,121600,121800,123300,125200,126400,127000,127400,128300,129100
4,40326,Phoenix,AZ,Phoenix,Maricopa,5,87200.0,87700.0,88200.0,88400.0,...,183800,185300,186600,188000,189100,190200,191300,192800,194500,195900


In [5]:
def get_list_of_university_towns():
    '''Returns a DataFrame of towns and the states they are in from the 
    university_towns.txt list. The format of the DataFrame should be:
    DataFrame( [ ["Michigan", "Ann Arbor"], ["Michigan", "Yipsilanti"] ], 
    columns=["State", "RegionName"]  )
    
    The following cleaning needs to be done:
    
    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''
    
    university_town = pd.read_table('university_towns.txt', names = {'University Town'})
    # university_town.head()

    State = []
    RegionName = []
    for index, row in university_town.iterrows():
        if(row.str.contains('edit').bool()):
            currentState = row[0]
    #         print("currentState", currentState)

        else:
    #         print("appending current state and region name")
            State.append(currentState)
            RegionName.append(row[0])
    #         print(row[0])

    df = pd.DataFrame({'State': State, 'RegionName': RegionName})
    
   
    
    df['State'] = df.State.str.split('[').str[0]
    df['RegionName'] = df.RegionName.str.split('(').str[0]
    
    df['State'] = df.State.map(lambda x: x.strip())
    df['RegionName'] = df.RegionName.map(lambda x: x.strip())
    
    
    df.reset_index()
    
    cols = df.columns.tolist()
    cols = cols[-1:] + cols[:-1]
    df = df[cols]
    return df


# print(get_list_of_university_towns())


             State                    RegionName
0          Alabama                        Auburn
1          Alabama                      Florence
2          Alabama                  Jacksonville
3          Alabama                    Livingston
4          Alabama                    Montevallo
5          Alabama                          Troy
6          Alabama                    Tuscaloosa
7          Alabama                      Tuskegee
8           Alaska                     Fairbanks
9          Arizona                     Flagstaff
10         Arizona                         Tempe
11         Arizona                        Tucson
12        Arkansas                   Arkadelphia
13        Arkansas                        Conway
14        Arkansas                  Fayetteville
15        Arkansas                     Jonesboro
16        Arkansas                      Magnolia
17        Arkansas                    Monticello
18        Arkansas                  Russellville
19        Arkansas  

In [6]:
def get_recession_start():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 7,  parse_cols = "E:G") 
    GDP = GDP.rename(columns = {'Unnamed: 0':'Quarter','Unnamed: 1':'GDP in billions of current dollars', 'Unnamed: 2':'GDP in billions of chained 2009 dollars'})

    GDP['previous'] = GDP['GDP in billions of chained 2009 dollars'].shift(1)
    GDP['diff'] = GDP['GDP in billions of chained 2009 dollars'] - GDP['previous']

    countList = []
    for index, row in GDP.iterrows():
        if row['diff'] < 0:
            count += 1
            countList.append(count)        

        else:
            count = 0
            countList.append(count) 

    GDP['count'] = countList
    GDP['year'] = GDP['Quarter'].str[:4]

    GDP['year'] = GDP['year'].astype(int)

    return GDP.iloc[GDP[(GDP['count'] >= 2) & (GDP['year'] >= 2000)].index-1]['Quarter'].iloc[0]

# print(get_recession_start())

2008q3


In [7]:
       
# GDP = pd.read_excel("gdplev.xls", skiprows = 7,  parse_cols = "E:G") 
# GDP = GDP.rename(columns = {'Unnamed: 0':'Quarter','Unnamed: 1':'GDP in billions of current dollars', 'Unnamed: 2':'GDP in billions of chained 2009 dollars'})

# GDP['previous'] = GDP['GDP in billions of chained 2009 dollars'].shift(1)
# GDP['diff'] = GDP['GDP in billions of chained 2009 dollars'] - GDP['previous']
# GDP[GDP['Quarter']>'2008q3']

,Quarter,GDP in billions of current dollars,GDP in billions of chained 2009 dollars,previous,diff
247,2008q4,14549.9,14577.0,14891.6,-314.6
248,2009q1,14383.9,14375.0,14577.0,-202.0
249,2009q2,14340.4,14355.6,14375.0,-19.4
250,2009q3,14384.1,14402.5,14355.6,46.9
251,2009q4,14566.5,14541.9,14402.5,139.4
252,2010q1,14681.1,14604.8,14541.9,62.9
253,2010q2,14888.6,14745.9,14604.8,141.1
254,2010q3,15057.7,14845.5,14745.9,99.6
255,2010q4,15230.2,14939.0,14845.5,93.5
256,2011q1,15238.4,14881.3,14939.0,-57.7


In [8]:
def get_recession_end():
    '''Returns the year and quarter of the recession end time as a 
    string value in a format such as 2005q3'''
       
    GDP = pd.read_excel("gdplev.xls", skiprows = 7,  parse_cols = "E:G") 
    GDP = GDP.rename(columns = {'Unnamed: 0':'Quarter','Unnamed: 1':'GDP in billions of current dollars', 'Unnamed: 2':'GDP in billions of chained 2009 dollars'})

    GDP['previous'] = GDP['GDP in billions of chained 2009 dollars'].shift(1)
    GDP['diff'] = GDP['GDP in billions of chained 2009 dollars'] - GDP['previous']

    countList = []
    for index, row in GDP.iterrows():
        if row['diff'] < 0:
            count += 1
            countList.append(count)        

        else:
            count = 0
            countList.append(count) 

    GDP['count'] = countList
    GDP['year'] = GDP['Quarter'].str[:4]
    GDP['year'] = GDP['year'].astype(int)

    endList = []
    GDP['countPrevious'] = GDP['count'].shift(1)
    GDP['countNext'] = GDP['count'].shift(-1)

    for index, row in GDP.iterrows():
        if ((row['count']>0) & (row['countNext'] < row['count']) & (row['countPrevious'] < row['count'])):
            endList.append('End')
        else:
            endList.append('other')
     
    GDP['EndingList']  = endList
    
    
    return  GDP.iloc[GDP[(GDP['EndingList'] == 'End') & (GDP['Quarter'] >= get_recession_start())].index+2]['Quarter'].iloc[0]

# print(get_recession_end())

2009q4


In [9]:
def get_recession_bottom():
    '''Returns the year and quarter of the recession bottom time as a 
    string value in a format such as 2005q3'''
    
    GDP = pd.read_excel("gdplev.xls", skiprows = 7,  parse_cols = "E:G") 
    GDP = GDP.rename(columns = {'Unnamed: 0':'Quarter','Unnamed: 1':'GDP in billions of current dollars', 'Unnamed: 2':'GDP in billions of chained 2009 dollars'})

    GDP['previous'] = GDP['GDP in billions of chained 2009 dollars'].shift(1)
    GDP['diff'] = GDP['GDP in billions of chained 2009 dollars'] - GDP['previous']

    countList = []
    for index, row in GDP.iterrows():
        if row['diff'] < 0:
            count += 1
            countList.append(count)        

        else:
            count = 0
            countList.append(count) 

    GDP['count'] = countList
    GDP['year'] = GDP['Quarter'].str[:4]
    GDP['year'] = GDP['year'].astype(int)

    recessionList = []
    GDP['countPrevious'] = GDP['count'].shift(1)
    GDP['countNext'] = GDP['count'].shift(-1)

    x = 0
    for index, row in GDP.iterrows():
        if (((row['countNext'] >= 2) or (row['countNext'] == 0))  & (row['countPrevious'] < row['count'])):
            recessionList.append(('recession')+str(x))
        else:
            x+=1
            recessionList.append('other')

    GDP['recessionList']  = recessionList
    GDPGroup = GDP.loc[GDP.groupby('recessionList')["GDP in billions of chained 2009 dollars"].idxmin()]

    return GDPGroup[GDPGroup['Quarter'] >= get_recession_start()]['Quarter'].iloc[0]

# print(get_recession_bottom())

2009q2


In [10]:
# pd.read_csv('City_Zhvi_AllHomes.csv').columns.tolist()

In [11]:
def convert_housing_data_to_quarters():
    '''Converts the housing data to quarters and returns it as mean 
    values in a dataframe. This dataframe should be a dataframe with
    columns for 2000q1 through 2016q3, and should have a multi-index
    in the shape of ["State","RegionName"].
    
    Note: Quarters are defined in the assignment description, they are
    not arbitrary three month periods.
    
    The resulting dataframe should have 67 columns, and 10,730 rows.
    '''
    city_all_homes = pd.read_csv('City_Zhvi_AllHomes.csv',usecols=range(51,251))
    city_all_homes = city_all_homes.groupby(pd.PeriodIndex(city_all_homes.columns, freq='q'), axis=1).mean()
    columns = list(pd.date_range(
    start="2000", end="2017", freq='Q').map(
    lambda x: "{:}q{:}".format(x.year, x.quarter))[:-1])
    
    city_all_homes.columns = columns
    city_all_homes_Region = pd.read_csv('City_Zhvi_AllHomes.csv', usecols = ["State","RegionName"])
    city_all_homes_final = city_all_homes_Region.merge(city_all_homes, how = 'inner', left_index=True, right_index = True)
    city_all_homes_final = city_all_homes_final.fillna(0)
    city_all_homes_final['State'] = city_all_homes_final['State'].map(states)

    city_all_homes_final = city_all_homes_final.set_index(["State", "RegionName"])
    return city_all_homes_final


# print(convert_housing_data_to_quarters())

                                                2000q1        2000q2  \
State                RegionName                                        
New York             New York             0.000000e+00  0.000000e+00   
California           Los Angeles          2.070667e+05  2.144667e+05   
Illinois             Chicago              1.384000e+05  1.436333e+05   
Pennsylvania         Philadelphia         5.300000e+04  5.363333e+04   
Arizona              Phoenix              1.118333e+05  1.143667e+05   
Nevada               Las Vegas            1.326000e+05  1.343667e+05   
California           San Diego            2.229000e+05  2.343667e+05   
Texas                Dallas               8.446667e+04  8.386667e+04   
California           San Jose             3.742667e+05  4.065667e+05   
Florida              Jacksonville         8.860000e+04  8.970000e+04   
California           San Francisco        4.305000e+05  4.644667e+05   
Texas                Austin               1.429667e+05  1.452667

In [12]:
def get_recession_start_before():
    '''Returns the year and quarter of the recession start time as a 
    string value in a format such as 2005q3'''
    GDP = pd.read_excel("gdplev.xls", skiprows = 7,  parse_cols = "E:G") 
    GDP = GDP.rename(columns = {'Unnamed: 0':'Quarter','Unnamed: 1':'GDP in billions of current dollars', 'Unnamed: 2':'GDP in billions of chained 2009 dollars'})

    GDP['previous'] = GDP['GDP in billions of chained 2009 dollars'].shift(1)
    GDP['diff'] = GDP['GDP in billions of chained 2009 dollars'] - GDP['previous']

    countList = []
    for index, row in GDP.iterrows():
        if row['diff'] < 0:
            count += 1
            countList.append(count)        

        else:
            count = 0
            countList.append(count) 

    GDP['count'] = countList
    GDP['year'] = GDP['Quarter'].str[:4]

    GDP['year'] = GDP['year'].astype(int)

    return GDP.iloc[GDP[(GDP['count'] >= 2) & (GDP['year'] >= 2000)].index-2]['Quarter'].iloc[0]

# print(get_recession_start_before())

2008q2


In [13]:
# # # test output type (different, p, better)

# res = run_ttest()
# print('Type test:', 'Passed' if type(
#     res) == tuple else 'Failed')
# print('test `better` item type:','Passed' if type(res[0]) == bool or type(res[0]) == np.bool_ else 'Failed' )
# print('`p` item type test:', 'Passed' if type(
#     res[1]) == np.float64 else 'Failed')
# print('`different` item type test:',
#       'Passed' if type(res[2]) == str else 'Failed')
# print('`different` item spelling test:', 'Passed' if res[2] in [
#       "university town", "non-university town"] else 'Failed')

Type test: Passed
test `better` item type: Passed
`p` item type test: Passed
`different` item type test: Passed
`different` item spelling test: Passed


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [14]:

##other method slow
# hdf = hdf.reset_index()

# hdf['unique'] = hdf['State'] + hdf['RegionName']
# ul['unique'] = ul['State'] + ul['RegionName']
# new = hdf.merge(ul, how = 'outer', left_on=['unique'], right_on=['unique'])

# notUni = new[new['State_x'].notnull() & new['State_y'].isnull()]
# notUni = notUni[['State_x',
#  'RegionName_x',
#  '2008q2',
#  '2008q3',
#  '2008q4',
#  '2009q1',
#  '2009q2']].rename(columns={'State_x': 'State', 'RegionName_x': 'RegionName'})
# notUni = notUni.reset_index(drop=True)
# notUni['PriceRatio'] = notUni[rec_start].div(notUni[rec_bottom])


# uni2 = new[new['State_y'].notnull() & new['State_x'].notnull()]
# uni2 = uni2[['State_x',
#  'RegionName_x',
#  '2008q2',
#  '2008q3',
#  '2008q4',
#  '2009q1',
#  '2009q2']].rename(columns={'State_x': 'State', 'RegionName_x': 'RegionName'})

# uni2 =  uni2.reset_index(drop=True)
# uni2['PriceRatio'] = uni2[rec_start].div(uni2[rec_bottom])


# scipy.stats.ttest_ind(university_towns['PriceRatio'],non_university_towns['PriceRatio'], nan_policy='omit')

In [31]:
def run_ttest():
    '''First creates new data showing the decline or growth of housing prices
    between the recession start and the recession bottom. Then runs a ttest
    comparing the university town values to the non-university towns values, 
    return whether the alternative hypothesis (that the two groups are the same)
    is true or not as well as the p-value of the confidence. 
    
    Return the tuple (different, p, better) where different=True if the t-test is
    True at a p<0.01 (we reject the null hypothesis), or different=False if 
    otherwise (we cannot reject the null hypothesis). The variable p should
    be equal to the exact p value returned from scipy.stats.ttest_ind(). The
    value for better should be either "university town" or "non-university town"
    depending on which has a lower mean price ratio (which is equivilent to a
    reduced market loss).'''
    
    import scipy
    hdf = convert_housing_data_to_quarters()
    rec_start=get_recession_start_before()
    rec_bottom = get_recession_bottom()
    ul = get_list_of_university_towns()

    hdf = hdf[hdf.columns[(hdf.columns >= rec_start) & (hdf.columns <= rec_bottom)]]
#     hdf = hdf.dropna()

    subset = ul[['State', 'RegionName']]
    tuple_list = [tuple(x) for x in subset.values]

    university_towns = hdf.loc[tuple_list]
    non_university_towns = hdf.loc[~hdf.index.isin(tuple_list)]
    university_towns['PriceRatio'] = university_towns[rec_start].div(university_towns[rec_bottom])
    non_university_towns['PriceRatio'] = non_university_towns[rec_start].div(non_university_towns[rec_bottom])

#     university_towns = university_towns.dropna()
#     non_university_towns = non_university_towns.dropna()
    
    university_towns = university_towns.fillna(0)
    non_university_towns = non_university_towns.fillna(0)

    statistic, pvalue = scipy.stats.ttest_ind(non_university_towns['PriceRatio'],university_towns['PriceRatio'], nan_policy='omit')

    #where
    different = pvalue <0.01 #boolean True or False
    p = pvalue #numpy.float64
    if university_towns['PriceRatio'].mean() > non_university_towns['PriceRatio'].mean(): 
        better = 'non-university town'
    else: 
        better = 'university town'

    return  (different, p, better)

# print(run_ttest())


(True, 4.0961452674119614e-208, 'university town')


/opt/conda/lib/python3.5/site-packages/ipykernel/__main__.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
